In [1]:
import pandas as pd
import csv
import re
import datetime

In [2]:
names = ["Speaker", "Speech", "date"]
scoParl = pd.read_csv(filepath_or_buffer= "Scots_Parl.csv", delimiter="*", quotechar="~", parse_dates=[2], infer_datetime_format=True, dayfirst=True, lineterminator=">", quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8', on_bad_lines='warn', names=names)

In [25]:
scoParl.head()

,Speaker,Speech,date
0,Interests Trish Godman (Oldest Committee Me...,I welcome everyone to the first meeting of ...,Thursday 9 June 2005
1,Alasdair Morgan (South of Scotland) (SNP):,I have no interests to declare.,Thursday 9 June 2005
2,Lord James Douglas-Hamilton (Lothians) (Con):,"To the best of my knowledge, I have no inter...",Thursday 9 June 2005
3,Scott Barrie (Dunfermline West) (Lab):,I have no interests to declare.,Thursday 9 June 2005
4,George Lyon (Argyll and Bute) (LD):,I have no interests to declare.,Thursday 9 June 2005


In [3]:
msps = pd.read_csv('Msps - final masterlist.csv', skip_blank_lines=True, na_filter=True, infer_datetime_format=True, parse_dates=[5, 6], index_col=0)

# msps.dropna(how = 'all', inplace=True)

In [4]:
#Master list with names as held in the 
msp_set = list(set(msps["Name"]))
msp_check = []
for i in range(len(msp_set)):
    holder = msp_set[i].lower()
    adjustment = holder.split()
    msp_check.append("\s*".join((adjustment[0], adjustment[1])))
    
    

def name_to_party(row, dataframe):
    #Have I found the name in the MSPS list
    is_msp = False
    options = []
    title = row["Speaker"]
    date = row["date"]
    for i in range(len(msp_check)):
        if bool(re.search(msp_check[i], title.lower())):
            options.append(msp_set[i])
            is_msp = True
    if is_msp == True and len(options) == 1:
        hold_row = msps.loc[(msps["Name"] == options[0]) & (msps["start date"] <= date) & (msps["end date"]> date),["Name", "Party", "Constituency or Region"]]
        if len(hold_row) == 1:
            return hold_row["Name"].to_list()[0], hold_row["Party"].to_list()[0], hold_row["Constituency or Region"].to_list()[0]
    
        elif len(hold_row) == 0:
            return title, "n/a", "n/a"
        
        else:
            return hold_row["Name"].iloc[0], "Manual input", "Manual input"
        
    
    elif is_msp == True and len(options) > 1:
        for msp in options:
            hold_row = msps.loc[(msps["Name"] == msp) & (msps["start date"] < date) & (msps["end date"]> date),["Name", "Party", "Constituency or Region"]]
            if len(hold_row) == 1:
                return hold_row["Name"].to_list()[0], hold_row["Party"].to_list()[0], hold_row["Constituency or Region"].to_list()[0]
            

        
    else:
        return title, "n/a", "n/a"
        
        
        
            
        
    
    

In [5]:
def text_tidier(text):
    contiguous_whitespace = " ( )+"
    subspace = " "
    
    holder = re.sub(contiguous_whitespace, subspace, text)
    newline_whitespace = "( )*\n( )*"
    subline = "\n"
    holder2 = re.sub(newline_whitespace, subline, holder)
    start_whitespace = "^\s*"
    start = ""
    new_text = re.sub(start_whitespace, start, holder2)
    return new_text

In [6]:
scoParl["date"].loc[scoParl["date"] == " ( Afternoon ) "] = " Wednesday 26 June 2002 "

In [7]:
regex = "^\s*$"
scempty = []
for i in range(len(scoParl)):
    if bool(re.search(regex, scoParl["Speech"].iloc[i])):
        scempty.append(i)
        
nowy_titles = []
indexes = []

for c in range(len(scempty)):
    no_text_title = scoParl.loc[scempty[c], ("Speaker")]
    next_title = scoParl.loc[scempty[c]+1, ("Speaker")]
    next_is_msp = False
    for d in range(len(msp_check)):
        if bool(re.search(msp_check[d], next_title.lower())):
            next_is_msp = True
            
    if next_is_msp != True:
        new_title = no_text_title +" " + next_title
        nowy_titles.append(new_title)
        indexes.append(scempty[c]+1)
        
scoParl.loc[indexes, ("Speaker")] = nowy_titles

scoParl.drop(labels = scempty, inplace = True)

In [19]:
len(scempty)

2764

In [8]:
changes = scoParl.apply(func=name_to_party, axis=1,result_type='expand', dataframe = msps)

In [9]:
new_speech = scoParl["Speech"].apply(func=text_tidier)

In [16]:
alpha = changes.loc[changes["Party"] == "Manual input required"].index


In [17]:
len(alpha)

0

In [68]:
beta = scoParl.loc[alpha].apply(func=name_to_party, axis=1,result_type='expand', dataframe = msps)

1
1
0
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [70]:
changes.loc[alpha] = beta

In [75]:
changes.shape, scoParl.shape

((1580123, 3), (1580123, 3))

In [ ]:
final_form = pd.DataFrame([changes, ])

In [76]:
changes

,0,1,2
0,Trish Godman,SLP,West Renfrewshire
1,Alasdair Morgan,SNP,South of Scotland
2,James Douglas-Hamilton,SCUP,Lothians
3,Scott Barrie,SLP,Dunfermline West
4,George Lyon,SLDP,Argyll and Bute
...,...,...,...
1582882,Michael Matheson,SNP,Falkirk West
1582883,Kevin Stewart,SNP,Aberdeen Central
1582884,Michael Matheson,SNP,Falkirk West
1582885,Kevin Stewart,SNP,Aberdeen Central


In [11]:
holdme = {
    0:"Speaker",
    1: "Party",
    2: "Constituency or Region"
}

In [79]:
changes.rename(columns = holdme)

,Speaker,Party,Constituency or Region
0,Trish Godman,SLP,West Renfrewshire
1,Alasdair Morgan,SNP,South of Scotland
2,James Douglas-Hamilton,SCUP,Lothians
3,Scott Barrie,SLP,Dunfermline West
4,George Lyon,SLDP,Argyll and Bute
...,...,...,...
1582882,Michael Matheson,SNP,Falkirk West
1582883,Kevin Stewart,SNP,Aberdeen Central
1582884,Michael Matheson,SNP,Falkirk West
1582885,Kevin Stewart,SNP,Aberdeen Central


In [82]:
changes["Date"] = scoParl["date"]

In [12]:
changes.rename(columns = holdme, inplace=True)

In [13]:
changes

,Speaker,Party,Constituency or Region
0,Trish Godman,SLP,West Renfrewshire
1,Alasdair Morgan,SNP,South of Scotland
2,James Douglas-Hamilton,SCUP,Lothians
3,Scott Barrie,SLP,Dunfermline West
4,George Lyon,SLDP,Argyll and Bute
...,...,...,...
1582882,Michael Matheson,SNP,Falkirk West
1582883,Kevin Stewart,SNP,Aberdeen Central
1582884,Michael Matheson,SNP,Falkirk West
1582885,Kevin Stewart,SNP,Aberdeen Central


In [14]:
hodat = pd.to_datetime(scoParl.loc[:, ("date")])

In [23]:
changes["Speech"] = new_speech 
changes["Date"] = hodat

In [25]:
changes.sort_values(["Date"], inplace=True, ignore_index=True)

In [29]:
changes.head(300)

,Speaker,Party,Constituency or Region,Speech,Date
0,Sir David Steel,No Affiliation,Lothians,Members have one minute remaining in which to ...,1999-05-12
1,Sir David Steel,No Affiliation,Lothians,In the first round of voting in the election o...,1999-05-12
2,Sir David Steel,No Affiliation,Lothians,The voting period in the first round of voting...,1999-05-12
3,Sir David Steel,No Affiliation,Lothians,In the second round of voting in the election ...,1999-05-12
4,Sir David Steel,No Affiliation,Lothians,The voting time for this round is now closed. ...,1999-05-12
...,...,...,...,...,...
295,Richard Lochhead,SNP,North East Scotland,"That was a speech, not an intervention. If Mr ...",1999-05-19
296,Trish Godman,SLP,West Renfrewshire,I shall speak against the amendment moved by D...,1999-05-19
297,Brian Adam,SNP,North East Scotland,Will Trish Godman give way?,1999-05-19
298,Richard Lochhead,SNP,North East Scotland,"Thank you, Mr Presiding Officer— [ Interruptio...",1999-05-19


In [30]:
for i in changes["Speech"]:
    if type(i) != str:
        print(i)

In [31]:
changes.to_csv(path_or_buf= "spFinal.csv", sep="*", quotechar="~", lineterminator=">", quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8', date_format="%Y%m%d")